# Exploring Pipelines and Wrapping Up Phase 3

## Why Pipeline?

Pipelines can keep our code neat and clean all the way from gathering & cleaning our data, to creating models & fine-tuning them!

**Advantages**: 
- Reduces complexity
- Convenient 
- Flexible 
- Can help prevent mistakes (like data leakage between train and test set) 

Easily integrate transformers and estimators, plus cross validation!

<img src="images/grid_search_cross_validation.png" alt="cross validation image from sklearn's documentation" width=500>

Why might CV be good in instances when we're doing things like searching for optimal hyperparameters...?

- 


In [1]:
# Imports
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, plot_confusion_matrix

import category_encoders as ce

In [2]:
# May need to install category_encoders if you'd like to use it:
# !conda install -c conda-forge category_encoders

Data source: https://www.kaggle.com/c/cat-in-the-dat-ii

In [ ]:
# Grab, then explore data


In [ ]:
# Exploring numeric cols


In [ ]:
# Exploring object cols


In [ ]:
# Exploring target distribution


In [ ]:
# Define our X and y


# and train test split - to create our val holdout set!


### Baseline Model

Let's find out how hard our problem is, by throwing things at it and seeing what sticks!

Biggest thing to think about - what types of columns need to be treated differently?

Reference: https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html

In [ ]:
# Set up how to treat our columns

# Want to grab numeric columns


# Then grab columns with fewer than 10 unique values


# Then grab columns with more than 10, since we won't OHE those
    

In [ ]:
# Check our work


In [ ]:
# Now, set up the preprocessing steps for each type of col


In [ ]:
# Put together our preprocessor using a Column Transformer


In [ ]:
# Just out of curiosity, let's see what this looks like 


In [ ]:
# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.


In [ ]:
# Cross validate


Evaluate:

- 


### Try Another Model Type

In [ ]:
# Time for a new pipeline!


In [ ]:
# Cross validate


Evaluate:

- 


### Grid Search

In [ ]:
# Now how do we search using the pipeline? easily!
# First - new dt


In [ ]:
# Let's define some parameters to test!
# Note - need to include the name of what these params are for


In [ ]:
# Now put the pieces together

# Can take a WHILE

In [ ]:
# Edit this to explore the scores of the params
print(f"Avg Train ROC-AUC for ---: {results.cv_results_['mean_train_score'][0]:.5f}")
print("*" * 10)
print(f"Avg Test ROC-AUC for ---: {results.cv_results_['mean_test_score'][0]:.5f}")

### Validate

How does this perform on our val set?

In [ ]:
def evaluate(estimator, X_train, X_test, y_train, y_test, roc_auc='proba'):
    '''
    Evaluation function to show a few scores for both the train and test set
    Also shows a confusion matrix for the test set
    
    roc_auc allows you to set how to calculate the roc_auc score: 
    'dec' for decision_function or 'proba' for predict_proba 
    If roc_auc == 'skip', then it ignores calculating the roc_auc_score
    '''
    # grab predictions
    train_preds = estimator.predict(X_train)
    test_preds = estimator.predict(X_test)
    
    # output needed for roc_auc_score
    if roc_auc == 'skip': # skips calculating the roc_auc_score
        train_out = False
        test_out = False
    elif roc_auc == 'dec': # not all classifiers have decision_function
        train_out = estimator.decision_function(X_train)
        test_out = estimator.decision_function(X_test)
    elif roc_auc == 'proba':
        train_out = estimator.predict_proba(X_train)[:, 1] # proba for the 1 class
        test_out = estimator.predict_proba(X_test)[:, 1]
    else:
        raise Exception("The value for roc_auc should be 'skip', 'dec' or 'proba'.")
    
    # print scores
    print("Train Scores")
    print("------------")
    print(f"Accuracy: {accuracy_score(y_train, train_preds)}")
    print(f"F1 Score: {f1_score(y_train, train_preds)}")
    if type(train_out) == np.ndarray: # checking for roc_auc
        print(f"ROC-AUC: {roc_auc_score(y_train, train_out)}")
    print("----" * 5)
    print("Test Scores")
    print("-----------")
    print(f"Accuracy: {accuracy_score(y_test, test_preds)}")
    print(f"F1 Score: {f1_score(y_test, test_preds)}")
    if type(test_out) == np.ndarray:
        print(f"ROC-AUC: {roc_auc_score(y_test, test_out)}")
    
    # plot test confusion matrix
    plot_confusion_matrix(estimator, X_test, y_test, values_format=',.5g')
    plt.show()

In [ ]:
# Now use that function with the val set!


## Discussion Time!

What does it mean to decide between model types? What do we think best represents this data? What else might I want to explore or understand before deciding that?

- 


## Resources

Check out Aurélien Geron's notebook of an [end-to-end ml project](https://github.com/ageron/handson-ml2/blob/master/02_end_to_end_machine_learning_project.ipynb) on his GitHub repo based around his book [_Hands-On Machine Learning with Scikit-Learn and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems (2nd ed)_](https://www.oreilly.com/library/view/hands-on-machine-learning/9781491962282/)